# Segmentation by registration and feature extraction

In [1]:
import glob
import os
import os.path
import numpy as np
import pandas as pd
import maweight
import pickle
import logging
import tqdm

from config import manually_segmented_path, dissected_path
from config import output_path, leg_features_path, loin_features_path, belly_features_path
from config import groin_features_path, shoulder_features_path, body_features_path
from config import save_registered_images
from config import bin_width, bin_min, bin_max, threshold
from config import bin_width_body, bin_min_body, bin_max_body
from config import tmp_path, elastix_params, threads

image_format = "." + elastix_params['ResultImageFormat']

import warnings
warnings.filterwarnings('ignore')

LIMIT=None

# setting the logging format
FORMAT = '%(asctime)-15s %(clientip)s %(user)-8s %(message)s'
logging.basicConfig(format=FORMAT, level=logging.INFO)

Executables being used: /opt/elastix-5.1.0-linux/bin/elastix /opt/elastix-5.1.0-linux/bin/transformix


In [2]:
#delete all '_' character from filenames
def rename_files(files):
    new_files = [os.path.join(os.path.dirname(file), os.path.basename(file).replace("_", "")) for file in files]
    [os.rename(rfiles[0], rfiles[1]) for rfiles in zip(files, new_files)]
    return new_files

## Discovering files to process

In [3]:
manually_segmented_files = []
manually_segmented_files += sorted(glob.glob(os.path.join(manually_segmented_path, '*.nii')))
manually_segmented_files += sorted(glob.glob(os.path.join(manually_segmented_path, '*.nii.gz')))

manually_segmented_files = rename_files(manually_segmented_files)

manually_segmented_images= [f for f in manually_segmented_files if not any(f for i in ["leg", "loin", "belly", "groin", "shoulder", "body"] if i in f)]
manually_segmented_legs= [f for f in manually_segmented_files if 'leg' in f]
manually_segmented_loins= [f for f in manually_segmented_files if 'loin' in f]
manually_segmented_bellies= [f for f in manually_segmented_files if 'belly' in f]
manually_segmented_groins= [f for f in manually_segmented_files if 'groin' in f]
manually_segmented_shoulders= [f for f in manually_segmented_files if 'shoulder' in f]
manually_segmented_bodies= [f for f in manually_segmented_files if 'body' in f]

dissected_images = []
dissected_images = glob.glob(os.path.join(dissected_path, '*.nii'))
dissected_images += glob.glob(os.path.join(dissected_path, '*.nii.gz'))
dissected_images = sorted(dissected_images)

if LIMIT:
    dissected_images= dissected_images[:LIMIT]

dissected_images = rename_files(dissected_images)
print(f"Number of dissected images: {len(dissected_images)}")
print(f"Number of manually segmented images: {len(manually_segmented_images)}")
print(f"Number of leg masks: {len(manually_segmented_legs)}")
print(f"Number of loin masks: {len(manually_segmented_loins)}")
print(f"Number of belly masks: {len(manually_segmented_bellies)}")
print(f"Number of groin masks: {len(manually_segmented_groins)}")
print(f"Number of shoulder masks: {len(manually_segmented_shoulders)}")
print(f"Number of body masks: {len(manually_segmented_bodies)}")

Number of dissected images: 48
Number of manually segmented images: 5
Number of leg masks: 5
Number of loin masks: 5
Number of belly masks: 5
Number of groin masks: 5
Number of shoulder masks: 5
Number of body masks: 5


## Set Origin to Center of Mass

In [4]:
for d in dissected_images: 
    maweight.origin_to_center_of_mass(d)
for i, le, lo, be, g, s, bo in zip(manually_segmented_images, manually_segmented_legs, manually_segmented_loins, 
                      manually_segmented_bellies, manually_segmented_groins, manually_segmented_shoulders, 
                      manually_segmented_bodies):
    maweight.origin_to_center_of_mass(i, [le, lo, be, g, s, bo])

Processing image: data/dissected/p001.nii.gz and masks: []
Center of Mass of data/dissected/p001.nii.gz is already in origin.
Processing image: data/dissected/p002.nii.gz and masks: []
Center of Mass of data/dissected/p002.nii.gz is already in origin.
Processing image: data/dissected/p003.nii.gz and masks: []
Center of Mass of data/dissected/p003.nii.gz is already in origin.
Processing image: data/dissected/p004.nii.gz and masks: []
Center of Mass of data/dissected/p004.nii.gz is already in origin.
Processing image: data/dissected/p005.nii.gz and masks: []
Center of Mass of data/dissected/p005.nii.gz is already in origin.
Processing image: data/dissected/p006.nii.gz and masks: []
Center of Mass of data/dissected/p006.nii.gz is already in origin.
Processing image: data/dissected/p007.nii.gz and masks: []
Center of Mass of data/dissected/p007.nii.gz is already in origin.
Processing image: data/dissected/p008.nii.gz and masks: []
Center of Mass of data/dissected/p008.nii.gz is already in 

## Segmentation by Registration

In [5]:
for d in tqdm.tqdm(dissected_images):
    for i, le, lo, be, g, s, bo in zip(manually_segmented_images, manually_segmented_legs, manually_segmented_loins, 
                                       manually_segmented_bellies, manually_segmented_groins, 
                                       manually_segmented_shoulders, manually_segmented_bodies):
        output_leg= os.path.join(output_path, d.split(os.sep)[-1] + '_' + le.split(os.sep)[-1] + '_' + image_format)
        output_loin= os.path.join(output_path, d.split(os.sep)[-1] + '_' + lo.split(os.sep)[-1] + '_' + image_format)
        output_belly= os.path.join(output_path, d.split(os.sep)[-1] + '_' + be.split(os.sep)[-1] + '_' + image_format)
        output_groin= os.path.join(output_path, d.split(os.sep)[-1] + '_' + g.split(os.sep)[-1] + '_' + image_format)
        output_shoulder= os.path.join(output_path, d.split(os.sep)[-1] + '_' + s.split(os.sep)[-1] + '_' + image_format)
        output_body= os.path.join(output_path, d.split(os.sep)[-1] + '_' + bo.split(os.sep)[-1] + '_' + image_format)
        if save_registered_images:
            output_registered= os.path.join(output_path, d.split(os.sep)[-1] + '_' + i.split(os.sep)[-1] + '_' + image_format)
        else:
            output_registered= None
        if (any(not os.path.isfile(f) for f in [output_leg, output_loin, output_belly, output_groin, output_shoulder, output_body]) or 
                (save_registered_images and not os.path.isfile(output_registered))):
            maweight.register_and_transform(i, d, [le, lo, be, g, s, bo], [output_leg, output_loin, output_belly, output_groin, output_shoulder, output_body], 
                                            registered_image_path= output_registered, threads= threads, params= elastix_params, work_dir= tmp_path, verbose= 0)

100%|██████████████████████████████████████| 48/48 [26:55:48<00:00, 2019.76s/it]


## Extracting the features

In [6]:
def extract_features(dissected_images, manually_segmented_targets, bin_min, bin_max, bin_width):
    dataframes= []
    
    for d in tqdm.tqdm(dissected_images):
        #print('processing: %s' % d)
        
        fitted_masks= []
        
        for m in manually_segmented_targets:
            output_skull= os.path.join(output_path, d.split(os.sep)[-1] + '_' + m.split(os.sep)[-1]+ '_' + image_format)
            fitted_masks.append(output_skull)
        labels= [f.split(os.sep)[-1].split('_')[1] for f in fitted_masks]

        dataframes.append(maweight.extract_features_3d(d, fitted_masks, labels, bins=list(range(bin_min, bin_max+1, bin_width)), thresholds=[threshold]))
    dataframes= pd.concat(dataframes, axis=0, ignore_index=True)
    dataframes['filename']= dissected_images
    
    
    return dataframes

In [7]:
leg_features= extract_features(dissected_images, manually_segmented_legs, bin_min, bin_max, bin_width)
leg_features.to_csv(leg_features_path, index=False)

100%|███████████████████████████████████████████| 48/48 [19:06<00:00, 23.88s/it]


In [8]:
loin_features= extract_features(dissected_images, manually_segmented_loins, bin_min, bin_max, bin_width)
loin_features.to_csv(loin_features_path, index=False)

100%|███████████████████████████████████████████| 48/48 [18:40<00:00, 23.35s/it]


In [9]:
belly_features= extract_features(dissected_images, manually_segmented_bellies, bin_min, bin_max, bin_width)
belly_features.to_csv(belly_features_path, index=False)

100%|███████████████████████████████████████████| 48/48 [17:42<00:00, 22.14s/it]


In [10]:
groin_features= extract_features(dissected_images, manually_segmented_groins, bin_min, bin_max, bin_width)
groin_features.to_csv(groin_features_path, index=False)

100%|███████████████████████████████████████████| 48/48 [16:25<00:00, 20.54s/it]


In [11]:
shoulder_features= extract_features(dissected_images, manually_segmented_shoulders, bin_min, bin_max, bin_width)
shoulder_features.to_csv(shoulder_features_path, index=False)

100%|███████████████████████████████████████████| 48/48 [18:41<00:00, 23.37s/it]


In [12]:
body_features= extract_features(dissected_images, manually_segmented_bodies, bin_min_body, bin_max_body, bin_width_body)
body_features.to_csv(body_features_path, index=False)

100%|███████████████████████████████████████████| 48/48 [27:12<00:00, 34.01s/it]
